# POTUS Election 2016 Reddit data Analysis (1)

python=3.7

In [148]:
1

1

In [1]:
import pandas as pd
import networkx as nx
import seaborn as sns
import re
from joblib import dump, load
from tldextract import extract
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

### construct reply-comment network

In [149]:
df = load("16-pkl/16election_debunk_[lang=en][time=16.10-16.12].pkl")

In [150]:
df.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 30297 entries, 6793 to 78746321
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    30297 non-null  object 
 1   subreddit_id  30297 non-null  object 
 2   created_utc   30297 non-null  float64
 3   subreddit     30297 non-null  object 
 4   author        30297 non-null  object 
 5   id            30297 non-null  object 
 6   parent_id     30297 non-null  object 
 7   body          30297 non-null  object 
 8   lang          30297 non-null  object 
dtypes: float64(1), object(8)
memory usage: 2.3+ MB


In [ ]:
# for subbmission author mapping
comment_user_list = df[['parent_id']]
comment_user_list['id'] = comment_user_list_filt['parent_id'].str.replace('^t\d+_', '')
comment_user_list

In [16]:
import glob  

# file path contains all the files of reddit comment data
folder_path = '/mnt/data/wenlu/reddit-new/16-election-submission'  
  
# use glob library to find all the csv files
csv_files = glob.glob(f'{folder_path}/*.csv')  
  
# empty DataFrame to save all the data
all_data = pd.DataFrame()  

for file in csv_files:  
    df1 = pd.read_csv(file)  
    all_data = all_data.append(df1, ignore_index=True)  
print(all_data)

         author_flair_css_class  is_self  saved  ups  retrieved_on  archived  \
0                           MAL     True  False   23    1484291391     False   
1                          None     True  False   12    1484291391     False   
2                          None    False  False    4    1484291391     False   
3                          None    False  False    2    1484291391     False   
4                          None     True  False    4    1484291391     False   
...                         ...      ...    ...  ...           ...       ...   
25868372                   None    False  False    0    1480294928     False   
25868373                   None     True  False    0    1480294928     False   
25868374                   None    False  False    1    1480294928     False   
25868375                  Brynn    False  False    1    1480294928     False   
25868376                   None    False  False    2    1480294928     False   

                                       

In [138]:
all_data.info() # select the colums you need

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25868377 entries, 0 to 25868376
Data columns (total 36 columns):
 #   Column                  Dtype 
---  ------                  ----- 
 0   author_flair_css_class  object
 1   is_self                 bool  
 2   saved                   bool  
 3   ups                     int64 
 4   retrieved_on            int64 
 5   archived                bool  
 6   secure_media_embed      object
 7   author_flair_text       object
 8   locked                  bool  
 9   spoiler                 bool  
 10  over_18                 bool  
 11  title                   object
 12  url                     object
 13  link_flair_text         object
 14  score                   int64 
 15  thumbnail               object
 16  domain                  object
 17  permalink               object
 18  contest_mode            bool  
 19  edited                  object
 20  num_comments            int64 
 21  secure_media            object
 22  link_flair_css_c

In [42]:
submission_user_list = all_data['id'].to_list()

In [154]:
# create a dictionary of {id:author}
id_to_author = dict(zip(all_data['id'], all_data['author']))

# use .map to map the submission authors' information
comment_user_list['submission_author'] = comment_user_list['id'].map(id_to_author)

In [ ]:
#  map the comment authors' information
comment_id_to_author = dict(zip(df['parent_id'], df['author']))
comment_user_list['comment_author'] = comment_user_list['parent_id'].map(comment_id_to_author) 

In [ ]:
comment_user_list_notna = comment_user_list.dropna()
comment_user_list_notna

In [ ]:
# construct edge list that contains retweet-comment data
rt_list =comment_user_list_filt[comment_user_list['parent_id'].notnull()]
print(len(rt_list))

#author:the account name of the poster
#id:this item's identifier, e.g. "8xwlg"
#parent_id:ID of the thing this comment is a reply to, either the link or a comment in it
# edge list
#edge_list = pd.concat([comment_user_list['parent_id'], all_data['id'], axis=1)

edge_list = pd.DataFrame({  'source':rt_list['submission_author'], 'target':rt_list['comment_author']   })

32806


In [ ]:
edge_list_notnull = edge_list.dropna()

In [ ]:
edge_list

,source,target
6,u/gerroff2,u/PossiblyaShitposter
10,u/FroggerWithMyLife,u/Mastr_Blastr
17,u/ExpOriental,u/sjryan
26,u/petercfrank,u/Land_of_Enchantment
27,u/tiredddddaway,u/_samux_
...,...,...
128249,u/Joetheweirdo,u/descendus
128254,u/WanktheMank,u/Labore_Et_Constantia
128255,u/NoTaxesTrump,u/whywilson
128260,u/jk_211,u/JoeFabooche


In [ ]:
 #turn to the the gephi
D = nx.from_pandas_edgelist(edge_list, create_using=nx.DiGraph)    # 利用边的信息来构建有向图
nx.write_gexf(D, "/mnt/data/wenlu/reddit-new/16-pkl/new3_RTnetworks_of_16election_reddit[time=16.10.10-16.12.31].gexf")  # 保存有向图数据

In [ ]:
edge_list.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32806 entries, 6 to 128262
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   source  32801 non-null  object
 1   target  32806 non-null  object
dtypes: object(2)
memory usage: 768.9+ KB


#kcore = 2

In [ ]:
# remove the self cycle
D.remove_edges_from(nx.selfloop_edges(D))

# caculate the number of k_core=2
k_core_2 = nx.k_core(D, k=2)

In [ ]:
k_core_2.number_of_nodes()
k_core_2.number_of_edges()

2113

In [ ]:
in_degrees = dict(k_core_2.in_degree())

# sort the nodes by their degree
sorted_nodes_by_in_degree = sorted(in_degrees, key=in_degrees.get, reverse=True)

In [ ]:
for node in sorted_nodes_by_in_degree[:10]:
    print("Node:", node, "In-Degree:", in_degrees[node])

Node: u/[deleted] In-Degree: 108
Node: u/autotldr In-Degree: 86
Node: u/SnapshillBot In-Degree: 60
Node: u/PoliticsModeratorBot In-Degree: 32
Node: u/saku393939 In-Degree: 23
Node: u/trumpple In-Degree: 22
Node: u/bane187 In-Degree: 22
Node: u/Odawn In-Degree: 19
Node: u/anti_anti_adblock In-Degree: 18
Node: u/Panprometheus In-Degree: 17


In [ ]:
nx.write_gexf(k_core_2, "/mnt/data/wenlu/reddit-new/16-pkl/RTnetworks_of_KCORE2[topic=16election].gexf")  # 保存有向子图数据

## Communities analysis

In [ ]:
# gephi nodes dataframe
vexes = pd.read_csv("/mnt/data/wenlu/reddit-new/nodes-RC-16election(moduralarity=3).csv")
print(vexes.shape)
vexes.sample(5)

(27505, 6)


,Id,Label,modularity_class,indegree,outdegree,degree
6524,u/jonnythaiwongy9,u/jonnythaiwongy9,1552,1,0,1
15147,u/catholichelp,u/catholichelp,6041,2,0,2
12367,u/QuantoFarto,u/QuantoFarto,5425,1,0,1
22173,u/zzzebreh,u/zzzebreh,5967,1,0,1
2391,u/thepriceofaslave,u/thepriceofaslave,530,1,1,2


In [ ]:
# Top eight communities with the highest number of community vertices
# [(Community number, Percentage of number of community vertices)]
top8_list = [(5,0.1272),(168,0.0595),(173,0.0558),(5834,0.0445),(1628,0.0217),(559,0.0183),(4820,0.0159),(6041,0.0158)]

# get the top 8 ones
communities_top8 = []
for i in range(len(top8_list)):
    communities_top8.append(vexes[vexes['modularity_class'] == top8_list[i][0]])

# sort by their ingree
for x in communities_top8:
    x.sort_values(by='indegree', ascending=False, inplace=True)
    x.reset_index(inplace=True)